In [1]:
import pandas as pd
import numpy as np
import warnings


from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Visualization
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # to be able to display all columns of the DataFrames
warnings.filterwarnings("ignore")

In [2]:
df_coffee = pd.read_csv("data/coffee.csv")
df_country = pd.read_csv("data/c_country.csv")
df_continent = pd.read_csv("data/c_continent.csv")

In [3]:
# Renombrar columnas:

df_coffee.columns = df_coffee.columns.str.lower().str.replace('.', '_').str.replace(' ', '_')
df_country.columns = df_country.columns.str.lower().str.replace('.', '_').str.replace(' ', '_')
df_continent.columns = df_continent.columns.str.lower().str.replace('.', '_').str.replace(' ', '_')

In [4]:
df_coffee.columns

Index(['rec_id', 'species', 'continent_of_origin', 'country_of_origin',
       'harvest_year', 'expiration', 'variety', 'color', 'processing_method',
       'aroma', 'flavor', 'aftertaste', 'acidity', 'body', 'balance',
       'uniformity', 'clean_cup', 'sweetness', 'moisture', 'quakers',
       'category_one_defects', 'category_two_defects'],
      dtype='object')

In [5]:
df_continent.columns

Index(['country_of_origin', 'avg_aroma', 'avg_flavor', 'avg_aftertaste',
       'avg_acidity', 'avg_body', 'avg_balance', 'avg_uniformity',
       'avg_clean_cup', 'avg_sweetness', 'avg_moisture', 'avg_quakers',
       'avg_category_one_defects', 'avg_category_two_defects', 'rec_cnt'],
      dtype='object')

In [6]:
df_country.columns

Index(['country_of_origin', 'avg_aroma', 'avg_flavor', 'avg_aftertaste',
       'avg_acidity', 'avg_body', 'avg_balance', 'avg_uniformity',
       'avg_clean_cup', 'avg_sweetness', 'avg_moisture', 'avg_quakers',
       'avg_category_one_defects', 'avg_category_two_defects', 'rec_cnt'],
      dtype='object')

In [7]:
# Eliminar filas "duplicadas" en df_coffee:

df_clean = df_coffee.drop_duplicates(subset=['species', 'country_of_origin', 'harvest_year', 'expiration'])
df_clean

,rec_id,species,continent_of_origin,country_of_origin,harvest_year,expiration,variety,color,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,moisture,quakers,category_one_defects,category_two_defects
0,0,Arabica,Africa,Ethiopia,2014.0,04/03/16,NaN,Green,Washed / Wet,8.67,8.83,8.67,8.75,8.50,8.42,10.00,10.00,10.00,0.12,0,0,0
2,2,Arabica,North America,Guatemala,NaN,05/31/11,Bourbon,NaN,NaN,8.42,8.50,8.42,8.42,8.33,8.42,10.00,10.00,10.00,0.00,0,0,0
3,3,Arabica,Africa,Ethiopia,2014.0,03/25/16,NaN,Green,Natural / Dry,8.17,8.58,8.42,8.42,8.50,8.25,10.00,10.00,10.00,0.11,0,0,2
5,5,Arabica,South America,Brazil,2013.0,09/03/14,NaN,Bluish-Green,Natural / Dry,8.58,8.42,8.42,8.50,8.25,8.33,10.00,10.00,10.00,0.11,0,0,1
6,6,Arabica,South America,Peru,2012.0,09/17/13,Other,Bluish-Green,Washed / Wet,8.42,8.50,8.33,8.50,8.25,8.25,10.00,10.00,10.00,0.11,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,1333,Robusta,North America,United States,2012.0,02/28/13,Arusha,Green,Natural / Dry,7.92,7.50,7.42,7.42,7.42,7.42,9.33,10.00,7.58,0.00,0,0,0
1334,1334,Robusta,South America,Ecuador,2016.0,01/18/17,NaN,Blue-Green,NaN,7.75,7.58,7.33,7.58,5.08,7.83,10.00,10.00,7.75,0.00,0,0,1
1336,1336,Robusta,North America,United States,2014.0,12/23/15,NaN,NaN,Natural / Dry,7.33,7.33,7.17,7.42,7.50,7.17,9.33,9.33,7.42,0.00,0,0,6
1337,1337,Robusta,Asia,India,2013.0,08/25/15,NaN,Green,Natural / Dry,7.42,6.83,6.75,7.17,7.25,7.00,9.33,9.33,7.08,0.10,0,20,1


In [8]:
df_clean.isnull().sum()

rec_id                    0
species                   0
continent_of_origin       1
country_of_origin         1
harvest_year             44
expiration                0
variety                 157
color                   146
processing_method       108
aroma                     0
flavor                    0
aftertaste                0
acidity                   0
body                      0
balance                   0
uniformity                0
clean_cup                 0
sweetness                 0
moisture                  0
quakers                   0
category_one_defects      0
category_two_defects      0
dtype: int64

## Gestión de Nulos por columna: 

- Processing method: assignar categoría 'other'
- Variety: assignar categoría 'other'
- Color: imputar en función de los valores en las columnas 'one-two category defects' --> categoría amarillenta (two defects) y categoría marrón (one defect) --> si no hay valores categoría green
- Continent y Country --> drop
- Harvest year --> Moda

In [9]:
df_clean.dropna(subset=['country_of_origin', 'continent_of_origin'], inplace=True)

In [10]:
def imput_other (df, columns):
    for col in columns:
        df[col].fillna('Other', inplace=True)
    return df
columns_imput = ['variety', 'processing_method']

df_clean= imput_other(df_clean, columns_imput)

In [11]:
def imputar_color(row):
    if pd.isnull(row['color']):  
        if row['category_two_defects'] > 0:
            return 'Yellow'
        elif row['category_one_defects'] > 0:
            return 'Brown'
        else:
            return 'Green'
    else:
        return row['color']  


df_clean['color'] = df_clean.apply(imputar_color, axis=1)

In [12]:
# Verificar la moda de la columna completa (global)
moda_harvest_year = df_clean['harvest_year'].mode()[0]
print(f"Moda de Harvest.Year: {moda_harvest_year}")

# Imputar los valores nulos con la moda global
df_clean['harvest_year'].fillna(moda_harvest_year, inplace=True)

# Verificar si aún quedan nulos
print(df_clean['harvest_year'].isnull().sum())

Moda de Harvest.Year: 2014.0
0


In [14]:
df_clean.to_csv('data/coffee_clean.csv', index=False)

In [15]:
df_continent.to_csv('data/continent_clean.csv', index=False)

In [16]:
df_country.to_csv('data/country_clean.csv', index=False)